In [3]:
import pandas as pd

# Importer le DataFrame propre depuis le fichier CSV
df = pd.read_csv('data.csv')


/tmp/ipykernel_8799/285193129.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data.csv')


In [13]:
import pandas as pd

# Remplacer les valeurs non numériques par 0
df['NewExist'] = df['NewExist'].fillna(0)
df['UrbanRural'] = df['UrbanRural'].fillna(0)


# Convertir la colonne en type entier
df['NewExist'] = df['NewExist'].astype(int)
df['NewExist'].astype(int)

df['UrbanRural'] = df['UrbanRural'].astype(int)
df['UrbanRural'].astype(int)

df.dtypes

NAICS                 int64
ApprovalFY           object
Term                  int64
NewExist              int64
FranchiseCode         int64
UrbanRural            int64
RevLineCr            object
LowDoc               object
MIS_Status            int64
ApprovalDate_Year     int64
dtype: object

In [26]:
df.head(3)
df.drop(['ApprovalFY', 'ApprovalDate_Year'], axis =1)

,NAICS,Term,NewExist,FranchiseCode,UrbanRural,RevLineCr,LowDoc,MIS_Status
0,451120,84,2,1,0,N,N,0
1,722410,60,2,1,0,N,N,0
2,621210,180,1,1,0,N,N,0
3,0,60,1,1,0,N,N,0
4,0,240,1,1,0,N,N,0
...,...,...,...,...,...,...,...,...
897162,451120,60,1,1,0,NaN,NaN,0
897163,451130,60,1,1,0,Y,Y,0
897164,332321,108,1,1,0,N,N,0
897165,0,60,1,1,0,N,N,1


In [18]:
# Modèle

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Visualisation

from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

In [27]:
#separer dataset en features et target
X = df.drop('MIS_Status', axis=1)
y = df['MIS_Status']

In [22]:
df.columns

Index(['NAICS', 'ApprovalFY', 'Term', 'NewExist', 'FranchiseCode',
       'UrbanRural', 'RevLineCr', 'LowDoc', 'MIS_Status', 'ApprovalDate_Year'],
      dtype='object')

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

cat_col = ['RevLineCr', 'LowDoc', 'FranchiseCode' ]
num_col = ['NAICS', 'Term', 'NewExist', 'UrbanRural']

In [29]:
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])


preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, cat_col),
        ('num', num_transformer, num_col)
    ])

# Création du pipeline : prétraitement + modèle RandomForest
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', RandomForestClassifier())])

pipeline.fit(X_train, y_train)

# performance du modèle
score = pipeline.score(X_test, y_test)
print("Score du modèle :", score)


Score du modèle : 0.9202770935274251
